🧩 Cell 1 – Import + Konfigurasi Path & HPO Grid

In [3]:
# Cell 1: Import dan konfigurasi dasar

import os
import glob
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# === PATH DASAR (SESUAI STRUKTUR KAMU) ===

BASE_DIR = r"E:\0.TA_Teguh\dataset2"

# Folder raw (di dalamnya ada folder per subjek: Afi, Tsania, Tsamara)
RAW_DIR = BASE_DIR  # contoh: E:\0.TA_Teguh\dataset2\Afi\Jalan1.csv

# Folder output clustering dan torso (per subjek)
CLUSTER_DIR = os.path.join(BASE_DIR, "hasil_clustering")
TORSO_DIR   = os.path.join(BASE_DIR, "hasil_torso")  # dipakai nanti untuk branch torso

# Daftar subjek (folder) yang akan diproses
SUBJECTS = ["Afi", "Tsania", "Tsamara"]

# Pastikan folder hasil_clustering & hasil_torso + subfolder per subjek ada
for subj in SUBJECTS:
    os.makedirs(os.path.join(CLUSTER_DIR, subj), exist_ok=True)
    os.makedirs(os.path.join(TORSO_DIR,   subj), exist_ok=True)

# === MODE PERLAKUAN FRAME KURUS ===
# Jika titik dalam 1 frame < min_samples dan > 0 → mau dianggap 1 cluster kecil atau noise semua?
SMALL_FRAME_AS_SINGLE_CLUSTER = True

# === GRID PARAMETER UNTUK HPO (cari eps & min_samples terbaik) ===
EPS_CANDIDATES         = [0.25, 0.30, 0.32, 0.35, 0.40]
MIN_SAMPLES_CANDIDATES = [3, 5, 8]

# Berapa banyak file sampel per subjek untuk HPO
HPO_MAX_FILES_PER_SUBJECT = None


ModuleNotFoundError: No module named 'pandas'

🧩 Cell 2 – Utility: List File Mentah Per Subjek

In [ ]:
# Cell 2: Fungsi untuk mengambil semua file raw per subjek

def list_raw_files_for_subject(subject_name: str):
    """
    Mengembalikan list path penuh ke semua file CSV mentah
    untuk satu subjek, misal:
    E:\0.TA_Teguh\dataset2\Afi\Jalan1.csv, Jalan2.csv, dst.
    """
    subj_dir = os.path.join(RAW_DIR, subject_name)
    pattern = os.path.join(subj_dir, "*.csv")
    files = sorted(glob.glob(pattern))
    return files

# (Opsional) Cek cepat isi per subjek
for subj in SUBJECTS:
    files = list_raw_files_for_subject(subj)
    print(f"Subjek {subj}: {len(files)} file ditemukan")
    for f in files:
        print("  ", os.path.basename(f))
    print()


🧩 Cell 3 – DBSCAN untuk Satu Frame

In [ ]:
# Cell 3: DBSCAN untuk 1 frame

def run_dbscan_on_frame(df_frame: pd.DataFrame,
                        eps: float,
                        min_samples: int,
                        small_frame_as_single_cluster: bool = True) -> np.ndarray:
    """
    Menerima subset DataFrame untuk 1 frame (kolom x, y, z),
    mengembalikan array cluster_id untuk tiap titik di frame itu.
    
    - Jika jumlah titik >= min_samples → jalankan DBSCAN normal.
    - Jika 0 titik → kembalikan array kosong.
    - Jika 0 < jumlah titik < min_samples:
        - Jika small_frame_as_single_cluster=True → semua titik cluster 0.
        - Jika False → semua titik noise (-1).
    """
    if df_frame.empty:
        return np.array([], dtype=int)
    
    coords = df_frame[["x", "y", "z"]].values
    n_points = coords.shape[0]
    
    if n_points >= min_samples:
        db = DBSCAN(eps=eps, min_samples=min_samples)
        labels = db.fit_predict(coords)
        return labels.astype(int)
    else:
        if small_frame_as_single_cluster:
            labels = np.zeros(n_points, dtype=int)   # semua cluster 0
        else:
            labels = -np.ones(n_points, dtype=int)   # semua noise
        return labels


🧩 Cell 4 – Evaluasi Param DBSCAN untuk 1 File (HPO)

In [ ]:
# Cell 4: Evaluasi kualitas DBSCAN untuk 1 file (tanpa menyimpan CSV)

def evaluate_dbscan_params_on_file(input_path: str,
                                   eps: float,
                                   min_samples: int,
                                   small_frame_as_single_cluster: bool = True):
    """
    Jalankan DBSCAN per frame untuk satu file, lalu hitung metrik:
    - total_frames
    - frames_with_non_noise : frame yang punya cluster_id != -1
    - coverage              : frames_with_non_noise / total_frames
    - avg_clusters_per_frame: rata-rata jumlah cluster non-noise per frame
    - avg_non_noise_points  : rata-rata jumlah titik non-noise per frame
    """
    df = pd.read_csv(input_path)
    if "frame" not in df.columns:
        raise ValueError("Kolom 'frame' tidak ditemukan di file: " + input_path)
    
    frame_groups = df.groupby("frame").groups
    total_frames = len(frame_groups)
    
    frames_with_non_noise = 0
    clusters_count_list = []
    non_noise_points_list = []
    
    for frame_id, idx in frame_groups.items():
        df_frame = df.loc[idx]
        labels = run_dbscan_on_frame(
            df_frame,
            eps=eps,
            min_samples=min_samples,
            small_frame_as_single_cluster=small_frame_as_single_cluster
        )
        
        if labels.size == 0:
            clusters_count_list.append(0)
            non_noise_points_list.append(0)
            continue
        
        non_noise_mask = (labels != -1)
        non_noise_labels = labels[non_noise_mask]
        
        if non_noise_labels.size > 0:
            frames_with_non_noise += 1
            unique_clusters = np.unique(non_noise_labels)
            clusters_count_list.append(len(unique_clusters))
            non_noise_points_list.append(non_noise_labels.size)
        else:
            clusters_count_list.append(0)
            non_noise_points_list.append(0)
    
    coverage = frames_with_non_noise / total_frames if total_frames > 0 else 0.0
    avg_clusters_per_frame = np.mean(clusters_count_list) if clusters_count_list else 0.0
    avg_non_noise_points = np.mean(non_noise_points_list) if non_noise_points_list else 0.0
    
    return {
        "file": input_path,
        "eps": eps,
        "min_samples": min_samples,
        "total_frames": total_frames,
        "frames_with_non_noise": frames_with_non_noise,
        "coverage": coverage,
        "avg_clusters_per_frame": avg_clusters_per_frame,
        "avg_non_noise_points": avg_non_noise_points,
    }


🧩 Cell 5 – HPO: Coba Semua Kombinasi eps × min_samples di Beberapa File

In [ ]:
# Cell 5: HPO sederhana untuk DBSCAN di beberapa file sampel

def run_hpo_dbscan(sample_subjects=None,
                   max_files_per_subject=HPO_MAX_FILES_PER_SUBJECT):
    """
    Jalankan evaluasi DBSCAN untuk grid eps x min_samples
    di beberapa file sampel dari tiap subjek.
    
    sample_subjects: list subjek yang mau diikutkan (default: SUBJECTS)
    """
    if sample_subjects is None:
        sample_subjects = SUBJECTS
    
    results = []
    
    for subj in sample_subjects:
        raw_files = list_raw_files_for_subject(subj)
        if not raw_files:
            print(f"[HPO] Tidak ada file untuk subjek {subj}")
            continue
        
        sample_files = raw_files[:max_files_per_subject]
        print(f"[HPO] Subjek {subj}, sample files:")
        for f in sample_files:
            print("    ", os.path.basename(f))
        print()
        
        for fpath in sample_files:
            for eps in EPS_CANDIDATES:
                for ms in MIN_SAMPLES_CANDIDATES:
                    metrics = evaluate_dbscan_params_on_file(
                        input_path=fpath,
                        eps=eps,
                        min_samples=ms,
                        small_frame_as_single_cluster=SMALL_FRAME_AS_SINGLE_CLUSTER
                    )
                    metrics["subject"] = subj
                    results.append(metrics)
    
    if not results:
        print("Tidak ada hasil HPO (cek kembali folder dan SUBJECTS).")
        return None, None
    
    df_res = pd.DataFrame(results)
    
    # Skor sederhana:
    # score = coverage - penalty * |avg_clusters_per_frame - 1|
    penalty = 0.3
    df_res["score"] = df_res["coverage"] - penalty * (df_res["avg_clusters_per_frame"] - 1).abs()
    
    df_res_sorted = df_res.sort_values(by=["score", "coverage"], ascending=False)
    
    return df_res, df_res_sorted

# Contoh pemanggilan HPO (fase 1):
# df_hpo, df_hpo_sorted = run_hpo_dbscan()
# df_hpo_sorted.head(20)


🧩 Cell 6 – Set Parameter Final DBSCAN dari Hasil HPO

In [ ]:
# Cell 6: Set parameter DBSCAN FINAL berdasarkan hasil HPO

# Contoh: misalkan dari df_hpo_sorted kamu pilih eps=0.30 dan min_samples=5
DBSCAN_EPS_FINAL         = 0.30    # ganti sesuai hasil HPO-mu
DBSCAN_MIN_SAMPLES_FINAL = 5       # ganti sesuai hasil HPO-mu

print("Parameter DBSCAN final:")
print("  eps        =", DBSCAN_EPS_FINAL)
print("  min_samples=", DBSCAN_MIN_SAMPLES_FINAL)


🧩 Cell 7 – Proses 1 File: Tambah cluster_id dan Simpan ke hasil_clustering

In [ ]:
# Cell 7: Proses 1 file raw → simpan file clustering (dengan cluster_id)

def build_output_cluster_path(subject_name: str, input_file_path: str) -> str:
    """
    Dari path input, misal:
        E:\0.TA_Teguh\dataset2\Afi\Jalan1.csv
    hasilkan path output:
        E:\0.TA_Teguh\dataset2\hasil_clustering\Afi\clustering_Jalan1.csv
    """
    fname = os.path.basename(input_file_path)      # "Jalan1.csv"
    output_fname = f"clustering_{fname}"          # "clustering_Jalan1.csv"
    subj_out_dir = os.path.join(CLUSTER_DIR, subject_name)
    return os.path.join(subj_out_dir, output_fname)


def process_single_file_dbscan(input_path: str,
                               output_path: str,
                               eps: float,
                               min_samples: int,
                               small_frame_as_single_cluster: bool = True):
    """
    Membaca file CSV mentah (timestamp, frame, x, y, z, doppler, SNR),
    melakukan DBSCAN per frame, menambahkan kolom cluster_id,
    lalu menyimpannya ke output_path.
    """
    print(f"Memproses file: {input_path}")
    df = pd.read_csv(input_path)
    
    if "frame" not in df.columns:
        raise ValueError("Kolom 'frame' tidak ditemukan dalam file: " + input_path)
    
    cluster_ids = np.full(len(df), -1, dtype=int)  # default noise
    
    for frame_id, idx in df.groupby("frame").groups.items():
        df_frame = df.loc[idx]
        labels = run_dbscan_on_frame(
            df_frame,
            eps=eps,
            min_samples=min_samples,
            small_frame_as_single_cluster=small_frame_as_single_cluster
        )
        cluster_ids[list(idx)] = labels
    
    df["cluster_id"] = cluster_ids
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"  >> Disimpan ke: {output_path}")


🧩 Cell 8 – Jalankan DBSCAN untuk Semua Subjek & Semua File (Fase Produksi)

In [ ]:
# Cell 8: Jalankan DBSCAN final untuk semua subjek & semua file raw

def run_dbscan_for_all_subjects(eps_final: float,
                                min_samples_final: int,
                                small_frame_as_single_cluster: bool = True):
    """
    Menggunakan parameter final hasil HPO,
    jalankan DBSCAN untuk semua file raw semua subjek,
    dan simpan hasil ke folder hasil_clustering/<subjek>/clustering_JalanN.csv
    """
    for subj in SUBJECTS:
        print("=" * 60)
        print(f"Subjek: {subj}")
        print("=" * 60)
        
        raw_files = list_raw_files_for_subject(subj)
        if not raw_files:
            print(f"  Tidak ada file CSV untuk subjek {subj}")
            continue
        
        for raw_path in raw_files:
            out_path = build_output_cluster_path(subj, raw_path)
            process_single_file_dbscan(
                input_path=raw_path,
                output_path=out_path,
                eps=eps_final,
                min_samples=min_samples_final,
                small_frame_as_single_cluster=small_frame_as_single_cluster
            )
        print()

# Contoh pemanggilan (FASE 2, setelah HPO & set param final):
# run_dbscan_for_all_subjects(DBSCAN_EPS_FINAL, DBSCAN_MIN_SAMPLES_FINAL)


🧩 Cell 9 (Opsional) – Ringkasan Cepat 1 File Clustering

In [ ]:
# Cell 9 (opsional): Ringkasan hasil clustering satu file

def summarize_cluster_file(cluster_file_path: str, max_frames: int = 5):
    """
    Tampilkan ringkasan singkat hasil clustering:
    - jumlah baris total
    - distribusi cluster_id
    - beberapa frame awal: jumlah titik per cluster
    """
    df = pd.read_csv(cluster_file_path)
    print(f"File: {cluster_file_path}")
    print(f"Total baris: {len(df)}")
    print("Distribusi cluster_id:")
    print(df["cluster_id"].value_counts().sort_index())
    print()
    
    frame_ids = sorted(df["frame"].unique())[:max_frames]
    for f in frame_ids:
        sub = df[df["frame"] == f]
        print(f"Frame {f}: {len(sub)} titik")
        print(sub["cluster_id"].value_counts().sort_index())
        print("-" * 30)

# Contoh:
# contoh_file = r"E:\0.TA_Teguh\dataset2\hasil_clustering\Afi\clustering_Jalan1.csv"
# summarize_cluster_file(contoh_file)


🧩 Cell 10 (Opsional) – Visualisasi 2D Satu Frame

In [ ]:
# Cell 10 (opsional): Visualisasi cluster untuk satu frame (2D)

def plot_clusters_for_frame(cluster_file_path: str, frame_id: int):
    """
    Plot 2D (x vs y) titik untuk satu frame tertentu, diwarnai berdasarkan cluster_id.
    Noise (-1) → abu-abu.
    """
    df = pd.read_csv(cluster_file_path)
    df_f = df[df["frame"] == frame_id]
    
    if df_f.empty:
        print(f"Tidak ada data untuk frame {frame_id}")
        return
    
    cluster_ids = df_f["cluster_id"].unique()
    
    plt.figure(figsize=(6, 6))
    for cid in sorted(cluster_ids):
        sub = df_f[df_f["cluster_id"] == cid]
        if cid == -1:
            label = "noise (-1)"
            alpha = 0.3
        else:
            label = f"cluster {cid}"
            alpha = 0.8
        plt.scatter(sub["x"], sub["y"], label=label, alpha=alpha)
    
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"{os.path.basename(cluster_file_path)} - frame {frame_id}")
    plt.legend()
    plt.axis("equal")
    plt.grid(True)
    plt.show()

# Contoh:
# plot_clusters_for_frame(contoh_file, frame_id=10)
